# Logistic Regreesion
- 逻辑回归属于分类任务，二分类任务
- 逻辑回归采用交叉熵损失函数
  - PyTorch中交叉熵为nn.CrossEntropyLoss(), 包含softmax在里面: https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html?highlight=nn%20crossentropyloss#torch.nn.CrossEntropyLoss
    - torch.nn中的交叉熵为nn.LogSoftmax()+nn.NLLLoss()，前者为算出标签对应输入中的某一类的log形式的softmax（log后值域为负无穷到0）得分，后者为将该得分与label相加，求平均，取相反数（方便更新）。
  - 

In [1]:
import torch
import random
import numpy
import d2lzh_pytorch as d2dl

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # 三维作图  

In [2]:
# Generate the data
# since it is a classfication task, we need generate fake data from two distributions
num_examples = 10000
num_features = 3
x_0 = torch.tensor(np.random.normal(-1,1,size=(500,num_features)),dtype=torch.float32)
y_0 = torch.zeros(500)
x_1 = torch.tensor(np.random.normal(1,1,size=(500,num_features)),dtype=torch.float32)
y_1 = torch.ones(500)
#print(x_0)
#print(x_1)
features = torch.cat((x_0,x_1),0)
labels = torch.cat((y_0,y_1),0)
print(features.size(),labels.size())
#print(labels)


# Visualize the data distribution
def visualize_data_2d(features, labels):
    d2dl.use_svg_display()
    d2dl.set_figsize()
    #plt.scatter(features[:,dim],labels,1)
    plt.scatter(features.data.numpy()[:,0], features.data.numpy()[:,1], c=labels.data.numpy(), s=100, lw=0, cmap='RdYlGn')

def visualize_data_3d(features, labels):
    d2dl.use_svg_display()
    d2dl.set_figsize()
    ax = Axes3D(plt.figure())
    NumP = 50
    x = features.data.numpy()[:,0]
    y = features.data.numpy()[:,1]
    z = features.data.numpy()[:,2]
    ax.scatter(x,y,z,s=40,c='r',edgecolor='k',alpha=0.5)

#visualize_data_2d(features, labels)
#visualize_data_3d(features, labels)

# Data Loader & Data Batcher
batch_size = 100
from torch.utils import data as Data
dataset = Data.TensorDataset(features, labels)
data_iterator = Data.DataLoader(dataset, batch_size, shuffle=True)

for X,y in data_iterator:
    print(y)
    break

# Define Model
from torch import nn as nn
class Logistic_Regression_Model(nn.Module):
    def __init__(self, num_features):
        super(Logistic_Regression_Model, self).__init__()
        self.linear = nn.Linear(num_features,1)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        y = self.sigmoid(self.linear(x))
        return y

# instance the network
net = Logistic_Regression_Model(num_features)

# check model parameters
for param in net.parameters():
    print(param)

# define loss function
loss_func = nn.BCELoss()

# define the optimizator
from torch import optim as optim
optimizor = optim.SGD(net.parameters(),lr=0.03, momentum=0.9)

# Training process
# initialize the parameters
from torch.nn import init
init.normal_(net.linear.weight,mean=0,std=0.01)
init.constant_(net.linear.bias,val=0)

# train
num_epochs = 10
for epoch in range(num_epochs):
    for X,y in data_iterator:
        output = net(X)
        #print(output)
        loss = loss_func(output,y.view(-1,1))
        mask = output.ge(0.5).float()
        #mask.data = torch.tensor(1.0,dtype=torch.float32)
        #print('y',y)
        #print(mask)
        correct = (mask == y.view(-1,1)).sum()
        # print(correct)
        accuracy = correct / X.size(0)
        
        optimizor.zero_grad()
        loss.backward()
        optimizor.step()
    if (epoch+1) % 1 == 0:
        print('epoch {} loss is {:.4f} accuracy is {:.4f}\n'.format(epoch+1 , loss.item(), accuracy))

        


torch.Size([1000, 3]) torch.Size([1000])
tensor([1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1.,
        1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1.,
        0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 1.,
        0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1.,
        1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 0.,
        0., 1., 0., 1., 0., 0., 0., 1., 0., 0.])
Parameter containing:
tensor([[0.0040, 0.1731, 0.4241]], requires_grad=True)
Parameter containing:
tensor([0.0107], requires_grad=True)
epoch 1 loss is 0.3013 accuracy is 0.9600

epoch 2 loss is 0.1688 accuracy is 0.9500

epoch 3 loss is 0.1225 accuracy is 0.9500

epoch 4 loss is 0.0810 accuracy is 0.9800

epoch 5 loss is 0.0808 accuracy is 0.9700

epoch 6 loss is 0.1025 accuracy is 0.9600

epoch 7 loss is 0.1198 accuracy is 0.9600

epoch 8 loss is 0.0731 accuracy is 0.9600

epoch 9 loss is 0.1189 accurac

In [3]:
# import packages
import torch
from torch import nn as nn
from torch import optim as optim
from torch.nn import init

import random
import numpy
import d2lzh_pytorch as d2dl

In [4]:
# Hyperparameter Definition
num_features = 2
num_examples = 4
num_epochs = 20

# Generate Data
# since it is a classfication task, we need generate fake data from two distributions
features = torch.Tensor([[0.6,0.3], [1.0,2.5], [3.5,3.4], [4.0,5.3]])
labels = torch.Tensor([[0.], [0.], [1.], [1.]])
print(features.size(),labels.size())

# Load Data
# in PyTorch 0.4 Variable is disgarded, torch.Tensor combines the original function on Variable
X = features
y = labels

# Define Model
class Logistic_Regression_Model(nn.Module):
    def __init__(self, num_features):
        super(Logistic_Regression_Model, self).__init__()
        self.linear = nn.Linear(num_features,1)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        y = self.sigmoid(self.linear(x))
        return y

# instance the network
net = Logistic_Regression_Model(num_features)

# check model parameters
for param in net.parameters():
    print(param)

# define loss function
loss_func = nn.BCELoss()

# define the optimizator

optimizor = optim.SGD(net.parameters(),lr=0.03, momentum=0.9)

# Training process
# initialize the parameters
init.normal_(net.linear.weight,mean=0,std=0.01)
init.constant_(net.linear.bias,val=0)

# train
for epoch in range(num_epochs):
    output = net(X)
    #print(output)
    loss = loss_func(output,y.view(-1,1))
    mask = output.ge(0.5).float()
    correct = (mask == y.view(-1,1)).sum()
    # print(correct)
    accuracy = correct / X.size(0)
        
    optimizor.zero_grad()
    loss.backward()
    optimizor.step()
    if (epoch+1) % 1 == 0:
        print('epoch {} loss is {:.4f} accuracy is {:.4f}\n'.format(epoch+1 , loss.item(), accuracy))

        


torch.Size([4, 2]) torch.Size([4, 1])
Parameter containing:
tensor([[ 0.5628, -0.6212]], requires_grad=True)
Parameter containing:
tensor([-0.0577], requires_grad=True)
epoch 1 loss is 0.6794 accuracy is 0.5000

epoch 2 loss is 0.6528 accuracy is 0.5000

epoch 3 loss is 0.6151 accuracy is 0.5000

epoch 4 loss is 0.5830 accuracy is 0.5000

epoch 5 loss is 0.5648 accuracy is 0.5000

epoch 6 loss is 0.5590 accuracy is 0.5000

epoch 7 loss is 0.5600 accuracy is 0.5000

epoch 8 loss is 0.5620 accuracy is 0.5000

epoch 9 loss is 0.5613 accuracy is 0.5000

epoch 10 loss is 0.5560 accuracy is 0.5000

epoch 11 loss is 0.5456 accuracy is 0.5000

epoch 12 loss is 0.5308 accuracy is 0.5000

epoch 13 loss is 0.5130 accuracy is 0.5000

epoch 14 loss is 0.4938 accuracy is 0.5000

epoch 15 loss is 0.4753 accuracy is 0.7500

epoch 16 loss is 0.4589 accuracy is 0.7500

epoch 17 loss is 0.4458 accuracy is 0.7500

epoch 18 loss is 0.4359 accuracy is 0.7500

epoch 19 loss is 0.4282 accuracy is 1.0000

epoc

In [5]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Hyper-parameters 
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# MNIST dataset (images and labels)
train_dataset = torchvision.datasets.MNIST(root='/Users/yanzheyuan/coding/dataset_pytorch/', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='/Users/yanzheyuan/coding/dataset_pytorch/', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

# Logistic regression model
model = nn.Linear(input_size, num_classes)

# Loss and optimizer
# nn.CrossEntropyLoss() computes softmax internally
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        images = images.reshape(-1, 28*28)
 
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
 
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
 
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


Epoch [1/5], Step [100/600], Loss: 2.2546
Epoch [1/5], Step [200/600], Loss: 2.1804
Epoch [1/5], Step [300/600], Loss: 2.0667
Epoch [1/5], Step [400/600], Loss: 1.9755
Epoch [1/5], Step [500/600], Loss: 1.8809
Epoch [1/5], Step [600/600], Loss: 1.7587
Epoch [2/5], Step [100/600], Loss: 1.7316
Epoch [2/5], Step [200/600], Loss: 1.7220
Epoch [2/5], Step [300/600], Loss: 1.6436
Epoch [2/5], Step [400/600], Loss: 1.5120
Epoch [2/5], Step [500/600], Loss: 1.5390
Epoch [2/5], Step [600/600], Loss: 1.5118
Epoch [3/5], Step [100/600], Loss: 1.4409
Epoch [3/5], Step [200/600], Loss: 1.4115
Epoch [3/5], Step [300/600], Loss: 1.3483
Epoch [3/5], Step [400/600], Loss: 1.3421
Epoch [3/5], Step [500/600], Loss: 1.2782
Epoch [3/5], Step [600/600], Loss: 1.2524
Epoch [4/5], Step [100/600], Loss: 1.1588
Epoch [4/5], Step [200/600], Loss: 1.1887
Epoch [4/5], Step [300/600], Loss: 1.2150
Epoch [4/5], Step [400/600], Loss: 1.1469
Epoch [4/5], Step [500/600], Loss: 1.2337
Epoch [4/5], Step [600/600], Loss: